# Notes
try:
* log transformations 
* KNN


In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime
from sgcrf import SparseGaussianCRF
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.preprocessing import MinMaxScaler


%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
sns.set(style="darkgrid")

GCRF - https://github.com/dswah/sgcrfpy

In [2]:
df = pd.read_csv('data/post_eda_data_2017-2019', index_col=0)
df.head()

/Users/rab/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,created_date,closed_date,agency,complaint_type,descriptor,location_type,address_type,status,resolution_description,community_board,borough,open_data_channel_type,location,new_zip,total_population,median_income,median_age,minority_rep,hs_or_above,bach_or_above,poverty_rate,unemployment_rate,created_day_o_wk,created_month,created_day_o_yr,created_yr,created_time,created_hr,created_date_only,response_time
0,2017-07-09 23:26:34,2017-07-10 01:55:52,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,ADDRESS,Closed,The Police Department responded to the complai...,17 BROOKLYN,BROOKLYN,MOBILE,"(40.65087119116162, -73.93997156019773)",11203,75451,30301,39.8,0.961816,0.786099,0.234683,0.147213,0.064832,6,7,190,2017,23:26:34,23,2017-07-09,0.103681
1,2017-07-09 23:11:08,2017-07-10 07:36:59,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,ADDRESS,Closed,The Police Department responded to the complai...,11 BRONX,BRONX,PHONE,"(40.85829772629949, -73.85445497857519)",10469,73870,29338,40.2,0.892961,0.705336,0.249883,0.134557,0.087722,6,7,190,2017,23:11:08,23,2017-07-09,0.351285
2,2017-07-09 23:15:39,2017-07-10 03:14:15,NYPD,Noise - Residential,Loud Music/Party,Residential Building/House,ADDRESS,Closed,The Police Department responded to the complai...,06 BRONX,BRONX,PHONE,"(40.856369402043434, -73.89027597242314)",10458,83960,17907,29.0,0.916687,0.614682,0.143008,0.313134,0.132113,6,7,190,2017,23:15:39,23,2017-07-09,0.165694
3,2017-07-09 20:12:26,2017-07-09 20:33:15,NYPD,Noise - Residential,Loud Talking,Residential Building/House,ADDRESS,Closed,The Police Department responded to the complai...,09 MANHATTAN,MANHATTAN,ONLINE,"(40.82104731868181, -73.95526006866052)",10031,60254,25155,34.2,0.825987,0.699188,0.359176,0.210074,0.068703,6,7,190,2017,20:12:26,20,2017-07-09,0.014456
4,2017-07-09 13:31:16,2017-07-09 17:44:29,NYPD,Noise - Residential,Loud Talking,Residential Building/House,ADDRESS,Closed,The Police Department responded to the complai...,02 QUEENS,QUEENS,ONLINE,"(40.74024808519835, -73.92024203568081)",11377,86421,30227,40.0,0.790479,0.703111,0.286637,0.107048,0.040060,6,7,190,2017,13:31:16,13,2017-07-09,0.175845


In [3]:
df.shape

(4674961, 30)

In [4]:
df.created_date = pd.to_datetime(df.created_date)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4674961 entries, 0 to 15492090
Data columns (total 30 columns):
created_date              datetime64[ns]
closed_date               object
agency                    object
complaint_type            object
descriptor                object
location_type             object
address_type              object
status                    object
resolution_description    object
community_board           object
borough                   object
open_data_channel_type    object
location                  object
new_zip                   int64
total_population          int64
median_income             int64
median_age                float64
minority_rep              float64
hs_or_above               float64
bach_or_above             float64
poverty_rate              float64
unemployment_rate         float64
created_day_o_wk          int64
created_month             int64
created_day_o_yr          int64
created_yr                int64
created_time         

In [6]:
def add_previous_call_meds(df, n_days=14, col_name='d_back', target='response_time'):
    n_days_back = np.array(range(1, n_days+1))
    
    for n in n_days_back:
        col = str(n) + col_name
        
        df[col] = df[target].shift(n)
        
    df.dropna(inplace=True)
    
    return df

In [7]:
# create a list of unique agency values
# because it's nice to have things cleanly sorted let's sort it alphabetically
agencies = sorted(list(df.agency.unique()))
agencies

['DEP', 'DOB', 'DOHMH', 'DOT', 'DPR', 'DSNY', 'HPD', 'NYPD']

In [8]:
# iterate through agency dictionary and add a timeseries of mean response times as key

agency_ts={}

for a in agencies:
    agency_ts[a] = df[df['agency'] == a].groupby(by=df.created_date.dt.date).response_time.median()
    agency_ts[a] = agency_ts[a].to_frame()
    agency_ts[a] = add_previous_call_meds(agency_ts[a], n_days=60)

In [9]:
agency_ts['NYPD']

,response_time,1d_back,2d_back,3d_back,4d_back,5d_back,6d_back,7d_back,8d_back,9d_back,10d_back,11d_back,12d_back,13d_back,14d_back,15d_back,16d_back,17d_back,18d_back,19d_back,20d_back,21d_back,22d_back,23d_back,24d_back,25d_back,26d_back,27d_back,28d_back,29d_back,30d_back,31d_back,32d_back,33d_back,34d_back,35d_back,36d_back,37d_back,38d_back,39d_back,40d_back,41d_back,42d_back,43d_back,44d_back,45d_back,46d_back,47d_back,48d_back,49d_back,50d_back,51d_back,52d_back,53d_back,54d_back,55d_back,56d_back,57d_back,58d_back,59d_back,60d_back
created_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-02,0.120909,0.109959,0.125185,0.120220,0.101100,0.111829,0.116291,0.122830,0.107442,0.133472,0.123478,0.104329,0.114190,0.121019,0.121019,0.117315,0.114554,0.115255,0.110556,0.113924,0.124306,0.108530,0.125932,0.122488,0.134340,0.110833,0.116458,0.122650,0.125104,0.126447,0.105822,0.117813,0.106528,0.109531,0.120799,0.120255,0.110301,0.123669,0.118652,0.112523,0.113900,0.125185,0.138600,0.118542,0.116308,0.103206,0.102402,0.117159,0.125509,0.123258,0.115810,0.122512,0.125000,0.097106,0.116794,0.116852,0.131030,0.127222,0.133137,0.117674,0.104838
2017-03-03,0.114398,0.120909,0.109959,0.125185,0.120220,0.101100,0.111829,0.116291,0.122830,0.107442,0.133472,0.123478,0.104329,0.114190,0.121019,0.121019,0.117315,0.114554,0.115255,0.110556,0.113924,0.124306,0.108530,0.125932,0.122488,0.134340,0.110833,0.116458,0.122650,0.125104,0.126447,0.105822,0.117813,0.106528,0.109531,0.120799,0.120255,0.110301,0.123669,0.118652,0.112523,0.113900,0.125185,0.138600,0.118542,0.116308,0.103206,0.102402,0.117159,0.125509,0.123258,0.115810,0.122512,0.125000,0.097106,0.116794,0.116852,0.131030,0.127222,0.133137,0.117674
2017-03-04,0.110602,0.114398,0.120909,0.109959,0.125185,0.120220,0.101100,0.111829,0.116291,0.122830,0.107442,0.133472,0.123478,0.104329,0.114190,0.121019,0.121019,0.117315,0.114554,0.115255,0.110556,0.113924,0.124306,0.108530,0.125932,0.122488,0.134340,0.110833,0.116458,0.122650,0.125104,0.126447,0.105822,0.117813,0.106528,0.109531,0.120799,0.120255,0.110301,0.123669,0.118652,0.112523,0.113900,0.125185,0.138600,0.118542,0.116308,0.103206,0.102402,0.117159,0.125509,0.123258,0.115810,0.122512,0.125000,0.097106,0.116794,0.116852,0.131030,0.127222,0.133137
2017-03-05,0.102963,0.110602,0.114398,0.120909,0.109959,0.125185,0.120220,0.101100,0.111829,0.116291,0.122830,0.107442,0.133472,0.123478,0.104329,0.114190,0.121019,0.121019,0.117315,0.114554,0.115255,0.110556,0.113924,0.124306,0.108530,0.125932,0.122488,0.134340,0.110833,0.116458,0.122650,0.125104,0.126447,0.105822,0.117813,0.106528,0.109531,0.120799,0.120255,0.110301,0.123669,0.118652,0.112523,0.113900,0.125185,0.138600,0.118542,0.116308,0.103206,0.102402,0.117159,0.125509,0.123258,0.115810,0.122512,0.125000,0.097106,0.116794,0.116852,0.131030,0.127222
2017-03-06,0.117685,0.102963,0.110602,0.114398,0.120909,0.109959,0.125185,0.120220,0.101100,0.111829,0.116291,0.122830,0.107442,0.133472,0.123478,0.104329,0.114190,0.121019,0.121019,0.117315,0.114554,0.115255,0.110556,0.113924,0.124306,0.108530,0.125932,0.122488,0.134340,0.110833,0.116458,0.122650,0.125104,0.126447,0.105822,0.117813,0.106528,0.109531,0.120799,0.120255,0.110301,0.123669,0.118652,0.112523,0.113900,0.125185,0.138600,0.118542,0.116308,0.103206,0.102402,0.117159,0.125509,0.123258,0.115810,0.122512,0.125000,0.097106,0.116794,0.116852,0.131030
2017-03-07,0.120637,0.117685,0.102963,0.110602,0.114398,0.120909,0.109959,0.125185,0.120220,0.101100,0.111829,0.116291,0.122830,0.107442,0.133472,0.123478,0.104329,0.114190,0.121019,0.121019,0.117315,0.114554,0.115255,0.110556,0.113924,0.124306,0.108530,0.125932,0.122488,0.134340,0.110833,0.116458,0.122650,0.125104,0.126447,0.105822,0.117813,0.106528,0.109531,0.120799,0.120255,0.110301,0.123669,0.118652,0.112523,0.113900,0.125185,0.138600,0.118542,0.116308,0.103206,0.102402,0.117159,0.125509,0.123258,0.115810,0.122512,0.125000,0.097106,0.116794,0.116852
2017-03-08,0.110694,

In [10]:
# create timeseries df for each borough

In [11]:
boroughs = sorted(list(df.borough.unique()))
boroughs

['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND']

In [12]:
boro_ts = {}

for b in boroughs:
    boro_ts[b] = df[df['borough'] == b].groupby(by=df.created_date.dt.date).response_time.median()
    boro_ts[b] = boro_ts[b].to_frame()
    boro_ts[b] = add_previous_call_meds(boro_ts[b], n_days=60)

In [13]:
boro_ts['BRONX'].head(10)

,response_time,1d_back,2d_back,3d_back,4d_back,5d_back,6d_back,7d_back,8d_back,9d_back,10d_back,11d_back,12d_back,13d_back,14d_back,15d_back,16d_back,17d_back,18d_back,19d_back,20d_back,21d_back,22d_back,23d_back,24d_back,25d_back,26d_back,27d_back,28d_back,29d_back,30d_back,31d_back,32d_back,33d_back,34d_back,35d_back,36d_back,37d_back,38d_back,39d_back,40d_back,41d_back,42d_back,43d_back,44d_back,45d_back,46d_back,47d_back,48d_back,49d_back,50d_back,51d_back,52d_back,53d_back,54d_back,55d_back,56d_back,57d_back,58d_back,59d_back,60d_back
created_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-02,1.717963,1.664311,1.766534,1.812500,1.276458,0.354508,1.740914,1.252870,1.618241,1.642894,1.309595,0.309502,0.385961,3.579578,1.631557,1.761175,1.516609,2.206991,1.020556,0.937031,3.290278,4.154757,3.155486,1.497199,1.364850,0.878368,2.035532,3.163657,1.478189,1.658177,1.546030,1.966823,0.680012,1.095799,2.125735,1.155694,1.565000,1.613044,1.528744,0.398733,0.634664,2.843611,1.086111,1.573611,1.689022,1.720579,1.762581,2.282622,4.059722,1.129155,1.643750,2.810509,3.048206,2.800017,3.094329,3.513889,1.643403,1.472037,1.634207,1.165637,0.236157
2017-03-03,2.985394,1.717963,1.664311,1.766534,1.812500,1.276458,0.354508,1.740914,1.252870,1.618241,1.642894,1.309595,0.309502,0.385961,3.579578,1.631557,1.761175,1.516609,2.206991,1.020556,0.937031,3.290278,4.154757,3.155486,1.497199,1.364850,0.878368,2.035532,3.163657,1.478189,1.658177,1.546030,1.966823,0.680012,1.095799,2.125735,1.155694,1.565000,1.613044,1.528744,0.398733,0.634664,2.843611,1.086111,1.573611,1.689022,1.720579,1.762581,2.282622,4.059722,1.129155,1.643750,2.810509,3.048206,2.800017,3.094329,3.513889,1.643403,1.472037,1.634207,1.165637
2017-03-04,2.708831,2.985394,1.717963,1.664311,1.766534,1.812500,1.276458,0.354508,1.740914,1.252870,1.618241,1.642894,1.309595,0.309502,0.385961,3.579578,1.631557,1.761175,1.516609,2.206991,1.020556,0.937031,3.290278,4.154757,3.155486,1.497199,1.364850,0.878368,2.035532,3.163657,1.478189,1.658177,1.546030,1.966823,0.680012,1.095799,2.125735,1.155694,1.565000,1.613044,1.528744,0.398733,0.634664,2.843611,1.086111,1.573611,1.689022,1.720579,1.762581,2.282622,4.059722,1.129155,1.643750,2.810509,3.048206,2.800017,3.094329,3.513889,1.643403,1.472037,1.634207
2017-03-05,1.905718,2.708831,2.985394,1.717963,1.664311,1.766534,1.812500,1.276458,0.354508,1.740914,1.252870,1.618241,1.642894,1.309595,0.309502,0.385961,3.579578,1.631557,1.761175,1.516609,2.206991,1.020556,0.937031,3.290278,4.154757,3.155486,1.497199,1.364850,0.878368,2.035532,3.163657,1.478189,1.658177,1.546030,1.966823,0.680012,1.095799,2.125735,1.155694,1.565000,1.613044,1.528744,0.398733,0.634664,2.843611,1.086111,1.573611,1.689022,1.720579,1.762581,2.282622,4.059722,1.129155,1.643750,2.810509,3.048206,2.800017,3.094329,3.513889,1.643403,1.472037
2017-03-06,2.302836,1.905718,2.708831,2.985394,1.717963,1.664311,1.766534,1.812500,1.276458,0.354508,1.740914,1.252870,1.618241,1.642894,1.309595,0.309502,0.385961,3.579578,1.631557,1.761175,1.516609,2.206991,1.020556,0.937031,3.290278,4.154757,3.155486,1.497199,1.364850,0.878368,2.035532,3.163657,1.478189,1.658177,1.546030,1.966823,0.680012,1.095799,2.125735,1.155694,1.565000,1.613044,1.528744,0.398733,0.634664,2.843611,1.086111,1.573611,1.689022,1.720579,1.762581,2.282622,4.059722,1.129155,1.643750,2.810509,3.048206,2.800017,3.094329,3.513889,1.643403
2017-03-07,1.906701,2.302836,1.905718,2.708831,2.985394,1.717963,1.664311,1.766534,1.812500,1.276458,0.354508,1.740914,1.252870,1.618241,1.642894,1.309595,0.309502,0.385961,3.579578,1.631557,1.761175,1.516609,2.206991,1.020556,0.937031,3.290278,4.154757,3.155486,1.497199,1.364850,0.878368,2.035532,3.163657,1.478189,1.658177,1.546030,1.966823,0.680012,1.095799,2.125735,1.155694,1.565000,1.613044,1.528744,0.398733,0.634664,2.843611,1.086111,1.573611,1.689022,1.720579,1.762581,2.282622,4.059722,1.129155,1.643750,2.810509,3.048206,2.800017,3.094329,3.513889
2017-03-08,2.056863,

In [14]:
# create timeseries df for each complaint_type

In [15]:
complaint_types = sorted(list(df.complaint_type.unique()))
complaint_types

['Blocked Driveway',
 'Building/Use',
 'Damaged Tree',
 'Dirty Conditions',
 'ELECTRIC',
 'General Construction/Plumbing',
 'HEAT/HOT WATER',
 'Illegal Parking',
 'Missed Collection (All Materials)',
 'Noise',
 'Noise - Commercial',
 'Noise - Residential',
 'Noise - Street/Sidewalk',
 'Noise - Vehicle',
 'PAINT/PLASTER',
 'PLUMBING',
 'Rodent',
 'Sanitation Condition',
 'Sewer',
 'Sidewalk Condition',
 'Street Condition',
 'Street Light Condition',
 'Traffic Signal Condition',
 'UNSANITARY CONDITION',
 'Water System']

In [16]:
complaint_ts = {}

for c in complaint_types:
    complaint_ts[c] = df[df['complaint_type'] == c].groupby(by=df.created_date.dt.date).response_time.median()
    complaint_ts[c] = complaint_ts[c].to_frame()
    complaint_ts[c] = add_previous_call_meds(complaint_ts[c], n_days=60)

In [17]:
complaint_ts['Noise - Residential'].head()

,response_time,1d_back,2d_back,3d_back,4d_back,5d_back,6d_back,7d_back,8d_back,9d_back,10d_back,11d_back,12d_back,13d_back,14d_back,15d_back,16d_back,17d_back,18d_back,19d_back,20d_back,21d_back,22d_back,23d_back,24d_back,25d_back,26d_back,27d_back,28d_back,29d_back,30d_back,31d_back,32d_back,33d_back,34d_back,35d_back,36d_back,37d_back,38d_back,39d_back,40d_back,41d_back,42d_back,43d_back,44d_back,45d_back,46d_back,47d_back,48d_back,49d_back,50d_back,51d_back,52d_back,53d_back,54d_back,55d_back,56d_back,57d_back,58d_back,59d_back,60d_back
created_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-03-02,0.117280,0.098791,0.122645,0.117523,0.110683,0.126701,0.123519,0.118750,0.105891,0.123773,0.122784,0.103611,0.112014,0.112708,0.117095,0.123785,0.094444,0.102402,0.118316,0.124907,0.120654,0.097598,0.119184,0.114699,0.135370,0.113495,0.137315,0.117535,0.132755,0.121950,0.098322,0.107251,0.111759,0.109468,0.125179,0.128738,0.099525,0.125984,0.117801,0.128275,0.119745,0.119936,0.135903,0.129873,0.110747,0.105046,0.109456,0.125718,0.124942,0.118727,0.114294,0.118941,0.111499,0.109433,0.117876,0.113866,0.122315,0.115602,0.121973,0.119769,0.105376
2017-03-03,0.109902,0.117280,0.098791,0.122645,0.117523,0.110683,0.126701,0.123519,0.118750,0.105891,0.123773,0.122784,0.103611,0.112014,0.112708,0.117095,0.123785,0.094444,0.102402,0.118316,0.124907,0.120654,0.097598,0.119184,0.114699,0.135370,0.113495,0.137315,0.117535,0.132755,0.121950,0.098322,0.107251,0.111759,0.109468,0.125179,0.128738,0.099525,0.125984,0.117801,0.128275,0.119745,0.119936,0.135903,0.129873,0.110747,0.105046,0.109456,0.125718,0.124942,0.118727,0.114294,0.118941,0.111499,0.109433,0.117876,0.113866,0.122315,0.115602,0.121973,0.119769
2017-03-04,0.114537,0.109902,0.117280,0.098791,0.122645,0.117523,0.110683,0.126701,0.123519,0.118750,0.105891,0.123773,0.122784,0.103611,0.112014,0.112708,0.117095,0.123785,0.094444,0.102402,0.118316,0.124907,0.120654,0.097598,0.119184,0.114699,0.135370,0.113495,0.137315,0.117535,0.132755,0.121950,0.098322,0.107251,0.111759,0.109468,0.125179,0.128738,0.099525,0.125984,0.117801,0.128275,0.119745,0.119936,0.135903,0.129873,0.110747,0.105046,0.109456,0.125718,0.124942,0.118727,0.114294,0.118941,0.111499,0.109433,0.117876,0.113866,0.122315,0.115602,0.121973
2017-03-05,0.108872,0.114537,0.109902,0.117280,0.098791,0.122645,0.117523,0.110683,0.126701,0.123519,0.118750,0.105891,0.123773,0.122784,0.103611,0.112014,0.112708,0.117095,0.123785,0.094444,0.102402,0.118316,0.124907,0.120654,0.097598,0.119184,0.114699,0.135370,0.113495,0.137315,0.117535,0.132755,0.121950,0.098322,0.107251,0.111759,0.109468,0.125179,0.128738,0.099525,0.125984,0.117801,0.128275,0.119745,0.119936,0.135903,0.129873,0.110747,0.105046,0.109456,0.125718,0.124942,0.118727,0.114294,0.118941,0.111499,0.109433,0.117876,0.113866,0.122315,0.115602
2017-03-06,0.119560,0.108872,0.114537,0.109902,0.117280,0.098791,0.122645,0.117523,0.110683,0.126701,0.123519,0.118750,0.105891,0.123773,0.122784,0.103611,0.112014,0.112708,0.117095,0.123785,0.094444,0.102402,0.118316,0.124907,0.120654,0.097598,0.119184,0.114699,0.135370,0.113495,0.137315,0.117535,0.132755,0.121950,0.098322,0.107251,0.111759,0.109468,0.125179,0.128738,0.099525,0.125984,0.117801,0.128275,0.119745,0.119936,0.135903,0.129873,0.110747,0.105046,0.109456,0.125718,0.124942,0.118727,0.114294,0.118941,0.111499,0.109433,0.117876,0.113866,0.122315


# Regression Models

In [94]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso

scaler = MinMaxScaler()

In [65]:
rf_rg = RandomForestRegressor(n_estimators=100)
linear_rg = LinearRegression()
lasso_rg = Lasso()
elastic_rg = ElasticNet()

In [84]:
model_results = pd.DataFrame(columns=['model','data','r2_score'])
model_results

,model,data,r2_score


## Helper

In [85]:
# create a helper function that fits a model to training data
# captures performance metrics, and records them in our dataframe
def add_model_results(model, data, model_name, data_name, df=model_results):
    
    X = data.drop(columns=['response_time'])
    y = data.response_time
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model.fit(X_train, y_train)
    test_preds = model.predict(X_test)
    r2 = r2_score(y_test, test_preds)
    
    scores = [{'model': model_name, 'data': data_name, 
               'r2_score': round(r2, 3)}]
    
    df = model_results.append(scores, ignore_index=True)
    
    return df

In [97]:
# create a helper function that fits a model to training data
# captures performance metrics, and records them in our dataframe
def add_model_results2(model, data, model_name, data_name, 
                      df=model_results, transform=False, transform_type=None):
    
    if transform == True: 
        if transform_type == 'log':
            data = np.log(data)
        elif transform_type == 'min_max':
            data = scaler.fit_transform(data)
    
    X = data.drop(columns=['response_time'])
    y = data.response_time
    
    tscv = TimeSeriesSplit(n_splits=3)
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model.fit(X_train, y_train)
    test_preds = model.predict(X_test)
    r2 = r2_score(y_test, test_preds)
    
    scores = [{'model': model_name, 'data': data_name, 
               'r2_score': round(r2, 3)}]
    
    df = model_results.append(scores, ignore_index=True)
    
    return df

In [86]:
models = {'RandomForest':rf_rg, 'LinearReg':linear_rg, 'Lasso':lasso_rg, 'ElasticNet':elastic_rg}

## Agency Models

In [87]:

for m in models:
    for a in agencies:
        model_results = add_model_results(model=models[m],
                                          data=agency_ts[a],
                                          model_name=m,
                                          data_name=a)
    
model_results

,model,data,r2_score
0,RandomForest,DEP,0.429
1,RandomForest,DOB,-0.017
2,RandomForest,DOHMH,-0.081
3,RandomForest,DOT,0.334
4,RandomForest,DPR,0.134
5,RandomForest,DSNY,0.307
6,RandomForest,HPD,0.713
7,RandomForest,NYPD,0.059
8,LinearReg,DEP,0.481
9,LinearReg,DOB,0.076


## Borough Models

In [88]:
for m in models:
    for b in boroughs:
        model_results = add_model_results(model=models[m],
                                          data=boro_ts[b],
                                          model_name=m,
                                          data_name=b)
    
model_results

,model,data,r2_score
0,RandomForest,DEP,0.429
1,RandomForest,DOB,-0.017
2,RandomForest,DOHMH,-0.081
3,RandomForest,DOT,0.334
4,RandomForest,DPR,0.134
5,RandomForest,DSNY,0.307
6,RandomForest,HPD,0.713
7,RandomForest,NYPD,0.059
8,LinearReg,DEP,0.481
9,LinearReg,DOB,0.076


## Complaint Type Models 

In [89]:
for m in models:
    for c in complaint_ts:
        model_results = add_model_results(model=models[m],
                                          data=complaint_ts[c],
                                          model_name=m,
                                          data_name=c)
    
model_results

,model,data,r2_score
0,RandomForest,DEP,0.429
1,RandomForest,DOB,-0.017
2,RandomForest,DOHMH,-0.081
3,RandomForest,DOT,0.334
4,RandomForest,DPR,0.134
5,RandomForest,DSNY,0.307
6,RandomForest,HPD,0.713
7,RandomForest,NYPD,0.059
8,LinearReg,DEP,0.481
9,LinearReg,DOB,0.076


In [90]:
model_results[model_results['r2_score'] >.5]

,model,data,r2_score
6,RandomForest,HPD,0.713
10,LinearReg,DOHMH,0.794
14,LinearReg,HPD,0.730
18,Lasso,DOHMH,0.533
22,Lasso,HPD,0.675
26,ElasticNet,DOHMH,0.688
30,ElasticNet,HPD,0.703
58,RandomForest,HEAT/HOT WATER,0.640
61,RandomForest,Noise,0.585
72,RandomForest,Street Condition,0.525


## Log Transformed Models 

In [96]:
log_model_results = pd.DataFrame(columns=['model','data','r2_score'])
log_model_results

,model,data,r2_score


In [98]:
for m in models:
    for a in agencies:
        log_model_results = add_model_results2(model=models[m],
                                               data=agency_ts[a],
                                               model_name=m,
                                               data_name=a,
                                               df=log_model_results,
                                               transform=True,
                                               transform_type='log')

log_model_results 

,model,data,r2_score
0,RandomForest,DEP,0.429
1,RandomForest,DOB,-0.017
2,RandomForest,DOHMH,-0.081
3,RandomForest,DOT,0.334
4,RandomForest,DPR,0.134
5,RandomForest,DSNY,0.307
6,RandomForest,HPD,0.713
7,RandomForest,NYPD,0.059
8,LinearReg,DEP,0.481
9,LinearReg,DOB,0.076


## MinMax Results

In [99]:
minmax_model_results = pd.DataFrame(columns=['model','data','r2_score'])
minmax_model_results

,model,data,r2_score


In [100]:
for m in models:
    for a in agencies:
        minmax_model_results = add_model_results2(model=models[m],
                                               data=agency_ts[a],
                                               model_name=m,
                                               data_name=a,
                                               df=minmax_model_results,
                                               transform=True,
                                               transform_type='min_max')

minmax_model_results 

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [21]:
# tscv = TimeSeriesSplit(n_splits=3)

In [24]:
# for train_index, test_index in tscv.split(X):
#     X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [25]:
# X_train = X[:int(X.shape[0]*0.7)]
# X_test = X[int(X.shape[0]*0.7):]
# y_train = y[:int(X.shape[0]*0.7)]
# y_test = y[int(X.shape[0]*0.7):]

In [26]:
# X_train.shape[0] == y_train.shape[0]

True

# LSTM

https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [53]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

np.random.seed(175)

Using TensorFlow backend.


In [54]:
scaler = MinMaxScaler(feature_range=(0,1))

In [55]:
scaled_data = scaler.fit_transform(test_data)

# Demographic Prediction

In [56]:
demo_df = pd.read_csv('data/sr_vol_by_zip.csv')
demo_df

,new_zip,Blocked Driveway,Building/Use,Damaged Tree,Dirty Conditions,ELECTRIC,GENERAL CONSTRUCTION,General Construction/Plumbing,HEAT/HOT WATER,HEATING,Illegal Parking,Missed Collection (All Materials),NONCONST,Noise,Noise - Commercial,Noise - Residential,Noise - Street/Sidewalk,Noise - Vehicle,PAINT - PLASTER,PAINT/PLASTER,PLUMBING,Rodent,Sanitation Condition,Sewer,Sidewalk Condition,Street Condition,Street Light Condition,Traffic Signal Condition,UNSANITARY CONDITION,Water System
0,10001,928.0,259.0,103.0,1354.0,282.0,290.0,1997.0,2233.0,996.0,3658.0,621.0,270.0,9565.0,2752.0,6719.0,3240.0,880.0,170.0,361.0,706.0,667.0,1009.0,918.0,1217.0,4644.0,67.0,7.0,652.0,2476.0
1,10002,1070.0,580.0,341.0,1912.0,1077.0,1017.0,2125.0,8886.0,3486.0,7910.0,503.0,624.0,7832.0,11693.0,19113.0,6147.0,2675.0,738.0,1468.0,2408.0,1647.0,1442.0,1195.0,1911.0,5705.0,1429.0,2925.0,2311.0,2604.0
2,10003,687.0,329.0,470.0,1919.0,795.0,994.0,2432.0,6450.0,2980.0,4541.0,1090.0,527.0,11385.0,7949.0,10976.0,5703.0,1870.0,805.0,1183.0,1989.0,1165.0,1117.0,1381.0,1706.0,6879.0,258.0,177.0,1763.0,2720.0
3,10004,82.0,22.0,14.0,112.0,21.0,11.0,166.0,53.0,81.0,1339.0,44.0,10.0,1946.0,225.0,242.0,439.0,242.0,6.0,11.0,21.0,87.0,104.0,281.0,521.0,1448.0,454.0,692.0,24.0,316.0
4,10005,69.0,20.0,19.0,137.0,11.0,27.0,183.0,61.0,113.0,1010.0,45.0,7.0,2932.0,355.0,619.0,1216.0,381.0,5.0,5.0,30.0,61.0,85.0,160.0,460.0,1159.0,561.0,488.0,37.0,187.0
5,10006,77.0,14.0,2.0,62.0,11.0,13.0,221.0,156.0,33.0,545.0,37.0,12.0,2270.0,300.0,329.0,409.0,73.0,12.0,3.0,33.0,112.0,35.0,133.0,343.0,724.0,226.0,260.0,31.0,187.0
6,10007,212.0,53.0,23.0,300.0,15.0,31.0,543.0,162.0,51.0,2136.0,130.0,8.0,3183.0,498.0,983.0,1278.0,244.0,18.0,21.0,56.0,323.0,240.0,332.0,678.0,2478.0,565.0,1376.0,24.0,672.0
7,10009,959.0,330.0,368.0,1691.0,1140.0,1152.0,1811.0,8043.0,4656.0,3514.0,597.0,690.0,7030.0,11178.0,20139.0,5301.0,1609.0,836.0,1426.0,2512.0,1452.0,900.0,1004.0,1154.0,3711.0,117.0,44.0,2171.0,2473.0
8,10010,292.0,160.0,108.0,717.0,197.0,359.0,1156.0,2552.0,1181.0,2234.0,514.0,174.0,6282.0,2052.0,5425.0,1715.0,798.0,205.0,286.0,809.0,507.0,410.0,619.0,680.0,3022.0,93.0,19.0,942.0,1512.0
9,10011,926.0,323.0,512.0,2509.0,808.0,1070.0,2777.0,4821.0,2228.0,4306.0,1623.0,562.0,10883.0,5736.0,7626.0,3329.0,1510.0,661.0,1061.0,2102.0,1278.0,1278.0,1533.0,1801.0,6078.0,163.0,146.0,1518.0,3008.0


In [57]:
census = pd.read_csv('data/nyc_census_data.csv')
census

,Unnamed: 0,zip_code,total_population,median_income,median_age,minority_rep,hs_or_above,bach_or_above,poverty_rate,unemployment_rate
0,"ZCTA5 11222: Summary level: 860, zip code tabu...",11222,36492,52004,34.8,0.265757,0.868476,0.587467,0.108999,0.048261
1,"ZCTA5 11233: Summary level: 860, zip code tabu...",11233,76819,28652,33.0,0.930746,0.719315,0.251099,0.251502,0.062119
2,"ZCTA5 11235: Summary level: 860, zip code tabu...",11235,78128,27444,45.1,0.281180,0.796760,0.477827,0.187858,0.058965
3,"ZCTA5 11426: Summary level: 860, zip code tabu...",11426,20801,40693,38.6,0.719533,0.821058,0.377583,0.071791,0.043863
4,"ZCTA5 11427: Summary level: 860, zip code tabu...",11427,24037,32027,42.0,0.801889,0.765993,0.321651,0.077908,0.052045
5,"ZCTA5 11101: Summary level: 860, zip code tabu...",11101,30043,40222,34.1,0.675598,0.822557,0.480186,0.177789,0.074581
6,"ZCTA5 11205: Summary level: 860, zip code tabu...",11205,46064,29349,29.8,0.577327,0.785712,0.451093,0.259105,0.084369
7,"ZCTA5 11215: Summary level: 860, zip code tabu...",11215,70156,63578,36.0,0.330620,0.904308,0.754513,0.078094,0.039470
8,"ZCTA5 11219: Summary level: 860, zip code tabu...",11219,90036,19006,27.8,0.381103,0.619849,0.191950,0.289280,0.057774
9,"ZCTA5 11226: Summary level: 860, zip code tabu...",11226,100277,29366,34.7,0.878726,0.785564,0.283729,0.161850,0.071901


In [58]:
demo_df.iloc[[0],[1,]]

,Blocked Driveway
0,928.0
